Data Collection and Preprocessing

In [4]:
import yfinance as yf
import pandas as pd

In [12]:
stock_data = yf.download("DKNG", start="2020-01-01", end="2024-01-01")

# Save relevant features: Close price, Volume, Daily Returns
stock_data['Daily Return'] = stock_data['Close'].pct_change()
stock_data.dropna(inplace=True)

# Save the data for later use
stock_data.to_csv("draftkings_data.csv")

print(stock_data.head())

[*********************100%***********************]  1 of 1 completed

Price       Close   High     Low   Open   Volume Daily Return
Ticker       DKNG   DKNG    DKNG   DKNG     DKNG             
Date                                                         
2020-01-03  10.68  10.75  10.610  10.66   467600     0.000000
2020-01-06  10.80  10.90  10.692  10.75  2183500     0.011236
2020-01-07  10.90  10.95  10.870  10.89  1977000     0.009259
2020-01-08  11.26  11.35  10.884  10.94  2955700     0.033028
2020-01-09  12.25  12.39  11.340  11.43  8279300     0.087922


ARIMA Implementation


In [16]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

# Define a function to predict using ARIMA
def arima_model(data, order=(5, 1, 0)):
    # Train the ARIMA model on 'Close' prices
    model = ARIMA(data['Close'], order=order)
    model_fit = model.fit()
    
    # Forecast the next price
    forecast = model_fit.forecast(steps=1)[0]
    
    # Decision based on forecast
    current_price = data['Close'].iloc[-1]
    if forecast > current_price:
        return "Buy"
    else:
        return "Sell"

# Test the ARIMA model
decision = arima_model(stock_data)
print(f"ARIMA Decision: {decision}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


KeyError: 0